In [ ]:
import pandas as pd, pathlib, math, csv

def _load_and_normalize_datetime(df: pd.DataFrame, datetime_cols=("comment_published_at","video_publish_date")):
    
    for c in datetime_cols:
        if c in df.columns:
            s = pd.to_datetime(df[c], errors="coerce", utc=True)
            df[c] = s.dt.tz_convert("UTC").dt.tz_localize(None)
    return df

def export_parquet_to_csv(parquet_path, csv_path, chunksize=None, utf8_for_excel=True):
    """
    parquet_path -> csv_path
    - chunksize=None: load all then write once (fine for < 5–10M rows)
    - utf8_for_excel=True: write UTF-8 with BOM so Excel on Windows opens it correctly
    """
    p = pathlib.Path(parquet_path)
    if not p.exists():
        raise FileNotFoundError(p)
    df = pd.read_parquet(p)
    df = _load_and_normalize_datetime(df)
    encoding = "utf-8-sig" if utf8_for_excel else "utf-8"
    if not chunksize:
        df.to_csv(csv_path, index=False, encoding=encoding, quoting=csv.QUOTE_MINIMAL)
        print(f"Wrote CSV: {csv_path} | rows={len(df)}")
        return

    
    wrote_header = False
    for start in range(0, len(df), chunksize):
        end = start + chunksize
        df.iloc[start:end].to_csv(
            csv_path, index=False, mode="a",
            header=not wrote_header, encoding=encoding, quoting=csv.QUOTE_MINIMAL
        )
        wrote_header = True
    print(f"Wrote chunked CSV: {csv_path}")

def export_parquet_to_excel(parquet_path, excel_path, max_rows_per_sheet=1_000_000, engine="xlsxwriter"):
    """
    parquet_path -> .xlsx with multiple sheets if needed.
    Excel has a hard limit of 1,048,576 rows per sheet.
    """
    p = pathlib.Path(parquet_path)
    if not p.exists():
        raise FileNotFoundError(p)
    df = pd.read_parquet(p)
    df = _load_and_normalize_datetime(df)

    n = len(df)
    n_sheets = max(1, math.ceil(n / max_rows_per_sheet))
    with pd.ExcelWriter(excel_path, engine=engine) as xw:
        for i in range(n_sheets):
            start = i * max_rows_per_sheet
            end = min((i+1) * max_rows_per_sheet, n)
            sheet = f"part_{i+1}"
            df.iloc[start:end].to_excel(xw, sheet_name=sheet, index=False)
    print(f"Wrote Excel: {excel_path} | rows={n} | sheets={n_sheets}")


In [ ]:
import pandas as pd, pathlib, re

YEAR = 2010  

DATA_DIR = pathlib.Path("yt_climate_data")
CSV_DIR  = DATA_DIR / "csv"
XLSX_DIR = DATA_DIR / "excel"


CSV_DIR.mkdir(parents=True, exist_ok=True)
XLSX_DIR.mkdir(parents=True, exist_ok=True)


PARQUET  = DATA_DIR / f"climate_comments_{YEAR}_all.parquet"
CSV_OUT  = CSV_DIR  / f"climate_comments_{YEAR}_all_clean.csv"
XLSX_OUT = XLSX_DIR / f"climate_comments_{YEAR}_all_clean.xlsx"

if not PARQUET.exists():
    raise FileNotFoundError(f"Parquet not found: {PARQUET}")

df = pd.read_parquet(PARQUET)


for c in ("comment_published_at","video_publish_date"):
    if c in df.columns:
        s = pd.to_datetime(df[c], errors="coerce", utc=True)
        df[c] = s.dt.tz_convert("UTC").dt.tz_localize(None)


ILLEGAL = re.compile(r"[\x00-\x08\x0B\x0C\x0E-\x1F]")
def clean_cell(x):
    if pd.isna(x): return x
    s = str(x)
    s = ILLEGAL.sub(" ", s)
    s = s.replace("\r\n","\n").replace("\r","\n")
    if s.startswith(("=", "+", "-", "@")):
        s = "'" + s
    return s

for col in ["comment_text","video_title","channel_name","region_group"]:
    if col in df.columns:
        df[col] = df[col].map(clean_cell)


df.to_csv(CSV_OUT, index=False, encoding="utf-8-sig")
print(f"Wrote CSV → {CSV_OUT} | rows: {len(df)}")


engine = None
for eng in ("openpyxl","xlsxwriter"):
    try:
        __import__(eng); engine = eng; break
    except ImportError:
        continue
if engine is None:
    raise ImportError("Install 'openpyxl' or 'xlsxwriter' to write .xlsx files.")

sheet_name = f"comments_{YEAR}"[:31]  
with pd.ExcelWriter(XLSX_OUT, engine=engine) as xw:
    df.to_excel(xw, sheet_name=sheet_name, index=False)

print(f"Wrote Excel → {XLSX_OUT} | engine={engine}")


In [ ]:
from pathlib import Path
import pandas as pd

climate_dir = Path(r"C:\Users\Admin\Downloads\Papers we care about\yt_climate_data\excel")
files = sorted(list(climate_dir.glob("climate_comments_*_all_clean.xlsx")) + 
               list(climate_dir.glob("climate_comments_*_all_clean.csv")), key=lambda p: p.name)

total = 0
for p in files:
    if p.suffix.lower() == ".xlsx":
        df = pd.read_excel(p, engine="openpyxl")
    else:
        df = pd.read_csv(p, encoding="utf-8", on_bad_lines="skip")
    col = "comment_text" if "comment_text" in df.columns else None
    if not col:
        print(f"{p.name}: NO comment_text column")
        continue
    nonempty = df[col].astype(str).str.strip().ne("").sum()
    print(f"{p.name}: {nonempty:,} usable rows")
    total += nonempty
print("TOTAL (pre leak-guard):", f"{total:,}")
